In [1]:
! pip install tpot

     |████████████████████████████████| 87 kB 3.2 MB/s 
     |████████████████████████████████| 173.5 MB 31 kB/s 
     |████████████████████████████████| 160 kB 52.5 MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11952 sha256=293967971e229bbfaf8a96a13baa8d840c2b059d3b43e73482250cd7e3c6d548
  Stored in directory: /root/.cache/pip/wheels/e2/d2/79/eaf81edb391e27c87f51b8ef901ecc85a5363dc96b8b8d71e3
Successfully built stopit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [2]:
import tpot
print('tpot: %s' % tpot.__version__)

tpot: 0.11.7


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [18]:
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedStratifiedKFold
from tpot import TPOTClassifier
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn import preprocessing

In [6]:
drive.mount('/content/drive')

Mounted at /content/drive


In [47]:
df=pd.read_csv('drive/MyDrive/BacteriaFolder/bacteria.csv')
df=df.fillna(0)
df.head()

,Ethyl Acetate,Ethanol,"Propanoic acid, ethyl ester",2-Pentanone,Decane,Methyl Isobutyl Ketone,Amylene hydrate,"Butanoic acid, 2-methyl-, methyl ester",Isobutyl acetate,Methyl isovalerate,1-Propanol,Methyl thiolacetate,"Butanoic acid, 2-methyl-, ethyl ester",2-Hexanone,Ethyl isocyanide,"1-Propanol, 2-methyl-","2-Pentanol, 2-methyl-",2-Pentanol,"1-Butanol, 3-methyl-, acetate",1 - Undecene,1-Butanol,2-Heptanone,Dodecane,"1-Butanol, 3-methyl-",S-Methyl 3-methylbutanethioate,"2-Heptanone, 4,6-dimethyl-","3-Buten-1-ol, 3-methyl-","Thiocyanic acid, methyl ester",Acetoin,"1-Pentanol, 2-methyl-","Butanoic acid, 3-methyl-, 2-methylbutyl ester","2-Heptanol, 4-methyl-",2-Nonanone,Acetic acid,2-Nonanol,Pyrrole,"1H-Pyrrole, 2-methyl-","1-Heptanol, 2,4-dimethyl-,",Propanoic acid,"Propanoic acid, 2-methyl-",1-Octanol,2-Undecanone,"Benzoic acid, methyl ester",Butyrolactone,"Decanoic acid, ethyl ester","Acetic acid, decyl ester",2-Undecanol,"Butanoic acid, 3-methyl-",2-Dodecanone,1-Decanol,2-Tridecanone,"Dodecanoic acid, ethyl ester","1,4-Butanediol",Phenylethyl Alcohol,"Acetophenone, 2'-amino-",2-Tridecanol,Tetradecanal,1-Dodecanol,Methyl tetradecanoate,2-Pentadecanone,"Tetradecanoic acid, ethyl ester",Hexadecanal,n-Tridecan-1-ol,1-Tetradecanol,n-Pentadecanol,1-Hexadecanol,Indole,Strain
0,465374.0,1027715,0.0,1289650,800581,324424.0,73015,0.0,0.0,0.0,0.0,82772.0,1019655.0,0.0,0.0,139019,0.0,0.0,111465.0,0.0,0.0,56603,0.0,3097686,0.0,0.0,334841,0.0,375792,0.0,0.0,0.0,0.0,4016234.0,0.0,0.0,0.0,0.0,51090.0,76990.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4007188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SA_A
1,193151.0,1050974,0.0,504113,294680,189630.0,0,0.0,0.0,0.0,0.0,0.0,245397.0,0.0,0.0,219733,0.0,56594.0,0.0,0.0,0.0,0,0.0,2130908,0.0,0.0,153294,0.0,102975,0.0,0.0,0.0,0.0,1565576.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1691000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SA_A
2,403286.0,1850391,0.0,1169501,15,228163.0,73558,0.0,0.0,0.0,0.0,0.0,494407.0,0.0,0.0,196279,0.0,124321.0,0.0,0.0,0.0,85741,0.0,2769590,0.0,0.0,341899,0.0,173956,0.0,0.0,0.0,0.0,1952847.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2140149.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SA_A
3,129833.0,5140770,0.0,1926072,124282,0.0,188367,0.0,0.0,0.0,0.0,0.0,2198502.0,0.0,0.0,375976,197683.0,53549.0,0.0,0.0,0.0,0,0.0,5101478,0.0,0.0,86222,0.0,381264,0.0,0.0,0.0,0.0,11875584.0,0.0,0.0,0.0,0.0,111819.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4678421.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SA_A
4,117105.0,3422557,0.0,246751,0,0.0,0,0.0,0.0,0.0,0.0,0.0,960937.0,0.0,0.0,93132,66816.0,0.0,0.0,0.0,0.0,0,0.0,2349861,0.0,0.0,0,0.0,233731,0.0,0.0,0.0,0.0,11024381.0,0.0,0.0,0.0,0.0,90520.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3219328.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SA_A


In [48]:
# convert df to numpy array
df=df.to_numpy()

# split into training and test set
X = df[:, :-1] # for all but last column

# normalise X
# norm = np.linalg.norm(X)
# X = X/norm

# import the last column
raw_y = df[:, -1] 

# label encode the y values
le = preprocessing.LabelEncoder()
le.fit(raw_y)
y=le.transform(raw_y)

X=X.astype(dtype=int)
y=y.astype(dtype=int)




In [50]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

In [51]:
model = TPOTClassifier(generations=5, population_size=50, cv=cv, scoring='accuracy', verbosity=2, random_state=1, n_jobs=-1)

In [53]:
model.fit(X, y)

Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.811574074074074

Generation 2 - Current best internal CV score: 0.8305555555555555

Generation 3 - Current best internal CV score: 0.8305555555555555

Generation 4 - Current best internal CV score: 0.8305555555555555

Generation 5 - Current best internal CV score: 0.8305555555555555

Best pipeline: MLPClassifier(MinMaxScaler(input_matrix), alpha=0.0001, learning_rate_init=0.001)


TPOTClassifier(config_dict=None, crossover_rate=0.1,
               cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=10, random_state=1),
               disable_update_check=False, early_stop=None, generations=5,
               log_file=None, max_eval_time_mins=5, max_time_mins=None,
               memory=None, mutation_rate=0.9, n_jobs=-1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=50,
               random_state=1, scoring='accuracy', subsample=1.0, template=None,
               use_dask=False, verbosity=2, warm_start=False)

In [55]:
model.export('drive/MyDrive/BacteriaFolder/tpot_best_model.py')